In [744]:
import json
import pandas as pd
import time
import ccy 
from tqdm import tqdm_notebook as tqdm
import collections
from collections import Counter

In [6]:
from ripple_api import RippleDataAPIClient,RippleRPCClient
api = RippleDataAPIClient('https://data.ripple.com')
rpc = RippleRPCClient('http://s1.ripple.com:51234/')

# 0. Data

Read the data, run only once 

In [7]:
df = pd.read_csv("ripple-transactions-jan-2013-dec-2016.csv")

In [1157]:
df.head()

,Unnamed: 0,hash,sender,receiver,currency,amount,ledger,sourceTag,DestinationTag,9,10
0,0,5C0504B229CD8EE803B01D78868B8F4B595D8B1D03636B...,rfe8yiZUymRPx35BEwGjhfkaLmgNsTytxT,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,XRP,0.1,7227428,NaN,NaN,1,1402920660
1,1,56022C64D29974C116591563DC8A80547907BE38EEB134...,rfe8yiZUymRPx35BEwGjhfkaLmgNsTytxT,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,XRP,0.1,7224947,NaN,NaN,1,1402908860
2,2,CCA2EB48BC106C7F4692BF71819EC236C9E09E37BCF38C...,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,rhjyoR8rpLMKM47pUgTjdNuVDJ1CrYYBwT,XRP,2000.0,5541569,NaN,NaN,1,1394924250
3,3,423708E30D94335FE2858C11B3D57390669A20FCE6C485...,rMTzGg7nPPEMJthjgEBfiPZGoAM7MEVa1r,rsdbFqwyGYX5fSvAmMEXBH2hRyBT8QfcUS,XRP,2020.0,3333190,NaN,NaN,1,1384463290
4,4,C1E342D3168740197C7E6B84110CEF9A338EA0D0EF5697...,rLz6TEYTsSZivwgXssq8ZSmTsBSFTMCw7h,rffqWVgnv9bPKaWHHVUDMjdAEgab3LGjZ1,XRP,100.0,6965416,NaN,NaN,1,1401681860


In [8]:
data = []
with open('more-txs-from-missing-parsed.json') as f:
    for line in tqdm(list(f)):
        data.append(json.loads(line))
with open('final_txs_2017.json') as f:
    for line in tqdm(list(f)):
        data.append(json.loads(line)) 

In [1158]:
for txn in data[0:5]:
    print(json.dumps(txn,indent = 4))

{
    "fee": 1.1e-05,
    "hash": "C221FA5733D106AB6A0D941050FC6702874D6707028A0757525E7413A8B1235A",
    "sender": "rwMyB1diFJ7xqEKYGYgk9tKrforvTr33M5",
    "success": "tesSUCCESS",
    "actualReceiverAmount": "0.000001",
    "XRPAcc": [
        {
            "isIncr": false,
            "account": "rwMyB1diFJ7xqEKYGYgk9tKrforvTr33M5",
            "amount": 1.1999999998124622e-05
        },
        {
            "isIncr": true,
            "account": "rfbKLd1VLB3o6fpkhCJexckArjoMmBm2wG",
            "amount": 9.997747838497162e-07
        }
    ],
    "flags": "",
    "actualReceiverCurrency": "XRP",
    "receiver": "rfbKLd1VLB3o6fpkhCJexckArjoMmBm2wG",
    "date": "2014-01-26T18:13:10+00:00",
    "actualIssuerReceiver": "--"
}
{
    "fee": 1e-05,
    "hash": "3F7A1B3A771D477B03104E86CE73790BB8C8E516BEE680BB4AD9893444D21CF3",
    "sender": "rHb9CJAWyB4rj91VRWn96DkukG4bwdtyTh",
    "success": "tesSUCCESS",
    "actualReceiverAmount": "0.000001",
    "XRPAcc": [
        {
            "i

# 1.Transactions

## 1.1 First hop

In [ ]:
wallets_first_hop = collections.defaultdict(list)
for txn in tqdm(data):
    sender = txn["sender"]
    try:
        for link in txn["links"]:
            if link["node1"] == sender:
                wallets_first_hop[sender].append(link["node2"])
    except KeyError:
        continue 

In [6]:
wallet_two_most_hops = {}
count_unique_first_hop = 0
count_unique_first_hop_if_more_than_one_transaction = 0
for k in tqdm(list(wallets_first_hop.keys())):
    counter=collections.Counter(wallets_first_hop[k])
    wallet_two_most_hops[k] = counter.most_common(2)
    if(len(wallet_two_most_hops[k]) == 1):
        count_unique_first_hop += 1
        for a,b in wallet_two_most_hops[k]:
            if(b != 1):
                count_unique_first_hop_if_more_than_one_transaction += 1

In [7]:
print("There are " + str(count_unique_first_hop/len(wallet_two_most_hops)*100) + "% of wallets that uses always same first hop" +
      " and only " + str(count_unique_first_hop_if_more_than_one_transaction/len(wallet_two_most_hops)*100) + "% if we consider wallets that did more than 1 transaction")

There are 85.15420507567998% of wallets that uses always same first hop and only 40.51439626721292% if we consider wallets that did more than 1 transaction


In [8]:
first_hops = []
for k in tqdm(list(wallets_first_hop.keys())):
    for hop in list(set(wallets_first_hop[k])):
        if hop not in first_hops:
            first_hops.append(hop)

In [9]:
gateways = api.get_all_gateways()

In [10]:
gateways_wallet = []
for k in tqdm(list(gateways.keys())):
    for gateway in gateways[k]:
        gateways_wallet.append(gateway["account"])

In [11]:
inter = list(set(gateways_wallet) & set(first_hops))

In [12]:
print("From the first hops only "+ str(len(inter)/len(first_hops)*100) + "% are know gateways.")

From the first hops only 4.644412191582003% are know gateways.


## 1.2 No hops

In [13]:
all_senders = set()
sender_with_paths = set()
all_txn = set()
txn_with_paths = set()
for txn in tqdm(data):
    sender = txn["sender"]
    txn_hash = txn["hash"]
    all_senders.add(sender)
    all_txn.add(txn_hash)
    if "paths" in txn.keys():
        sender_with_paths.add(sender)  
        txn_with_paths.add(txn_hash)

In [14]:
sender_with_only_two_hops = list(all_senders - sender_with_paths)
txn_with_only_two_hops = list(all_txn - txn_with_paths)

In [15]:
print("We have "+ str(len(sender_with_only_two_hops)/len(all_senders)*100) + "% of senders that never use paths")

We have 87.98582218291993% of senders that never use paths


In [16]:
print("We have "+ str(len(txn_with_only_two_hops)/len(all_txn)*100) + "% of transactions that never use paths")

We have 78.80077420297265% of transactions that never use paths


## 1.3. Currency of direct transactions

In [989]:
currency_direct_txn_not_XRP = set()
count_direct_txn_not_XRP = 0
count_direct_txn_XRP = 0
for i in tqdm(range(len(data))):
    txn = data[i]
    if txn['success'] == 'tesSUCCESS':
        #direct
        if 'links' not in txn.keys():
            if 'senderCurrency' in txn.keys():
                if txn['senderCurrency'] != 'XRP':
                    currency_direct_txn_not_XRP.add(txn['senderCurrency'])
                    count_direct_txn_not_XRP += 1
            else:
                count_direct_txn_XRP += 1
        #direct (1 link) 
        else:
            if len(txn['links']) <= 1:
                if 'offers' not in txn.keys():
                    count_direct_txn_XRP += 1

In [990]:
print("There are", count_direct_txn_not_XRP/(count_direct_txn_not_XRP+count_direct_txn_XRP) * 100,"% of direct payments in other curencies that XRP")

There are 0.020702439469105882 % of direct payments in other curencies that XRP


In [991]:
gateways = api.get_all_gateways()
online_gateways = ['SnapSwap','Gatehub Fifth','Bitstamp','Payroutes','WisePass','The Rock Trading','RippleChina','DotPayco','Coinex','Ripple Fox','Lake BTC','Bitso','GBI','Gatehub','EXRP','BPG']
online_gateways_dict = collections.defaultdict(list)
for currency,list_gateways in tqdm(gateways.items()):
    for gateway in list_gateways:
        if gateway['name'] in online_gateways:
            online_gateways_dict[currency].append(gateway)

In [992]:
online_gateways_dict = dict(online_gateways_dict)

In [993]:
currency_direct_payments_not_offered_by_gateways = 0
real_currency_direct_payments_not_offered_by_gateways = 0
currency_direct_payments_not_offered_by_online_gateways = 0
real_currency_direct_payments_not_offered_by_online_gateways = 0
for currency in tqdm(currency_direct_txn_not_XRP):
    
    #all gateways
    if currency not in gateways.keys():
        currency_direct_payments_not_offered_by_gateways += 1
        if currency == ccy.currency(currency):
            real_currency_direct_payments_not_offered_by_gateways += 1

    #online gateways
    if currency not in online_gateways_dict.keys():
        currency_direct_payments_not_offered_by_online_gateways += 1
        if currency == ccy.currency(currency):
            real_currency_direct_payments_not_offered_by_online_gateways += 1        

In [1037]:
print("There are", currency_direct_payments_not_offered_by_gateways, "currency that appears in direct payments that are not offered by gateways, of which",real_currency_direct_payments_not_offered_by_gateways, "are real world currencies")
print("There are", currency_direct_payments_not_offered_by_online_gateways, "currency that appears in direct payments that are not offered by online gateways, of which",real_currency_direct_payments_not_offered_by_online_gateways, "are real world currencies")

There are 18 currency that appears in direct payments that are not offered by gateways, of which 0 are real world currencies
There are 20 currency that appears in direct payments that are not offered by online gateways, of which 0 are real world currencies


# 2. Accounts

## 2.1 Account creation (Too long using API)

In [17]:
query_params = dict(start="2012-01-01",end="2019-12-31",interval="week",family="metric",metrics="accounts_created",limit=1000)
created_accounts_per_week = api.get_stats(**query_params)

In [18]:
nb_accounts = 0
for week in tqdm(created_accounts_per_week["stats"]):
    nb_accounts += week["accounts_created"]

In [19]:
print("There are",nb_accounts,"accounts")

There are 1446292 accounts


In [ ]:
all_senders = set()
for txn in tqdm(data):
    all_senders.add(txn["sender"])

In [23]:
all_txn.update(set(df.hash.tolist()))

In [24]:
all_txn = list(all_txn_hashes)

In [28]:
nb_txn = len(all_txn)
nb_txn

27406875

Too long

In [ ]:
creation_txns = []
pbar = tqdm(total = nb_txn-1)
i=0
while(i < nb_txn):
    hsh = all_txn[i]
    transaction = api.get_transaction(hsh)
    if(list(transaction.keys()) == ['status', 'error']):
        time.sleep(int(transaction["error"].headers.get('Retry-After')) + 1)
    else:
        i+=1
        pbar.update(1)
        for affectedNode in transaction["transaction"]["meta"]["AffectedNodes"]:
            if "CreatedNode" in affectedNode.keys():
                if(affectedNode["CreatedNode"]["LedgerEntryType"] == "AccountRoot"):
                    creation_txns.append(hsh)
pbar.close()                    

In [15]:
creation_txns

['411790751304693B1ABA32443A762713CA49B67D9F995A4DA4E8FD296EDC7E40',
 '59E57301AD19F764468F3D6DA874071DDF1B40416CAD88D9F6B79673F2CAC608',
 '58A8919EBDD59FD4BEB8194D36FE09062F3247ABA2B9497DCEB56575EF3E3311',
 'EEEB8A00C0EA1687E95B94BDB39A98F827BE20EB6426CC6960DABA06CEFF1BF3',
 '40A65FEF00CB908A86D2EF2A858A7684E4BBB78739E9296E9551F68638A89E2B',
 'C112F9C5AB5FE01F7753C38EFADE2909228B27AB642DF286CA42DC6518DD2FB7',
 '2302E4FCB121561AA2DEDEE0116AD451A982993A0DB6731FDD1DA78B511DDB51',
 'EC283B2A36D56685539F1DA284EE1E9125580299968B0C7FFD73E6FCBD16F7EF',
 'E1373B461997CFFE390FC45B3F75BA99C6966A37098638A6278F9EE5EEFB3891',
 '4AA7EFDABA33F1552FF4C536EAE65CBA605D7FA61CE765F9D9E79C8444FAF3B3',
 'FABD792DB5B44B70087FE5D1DA2E6D82B802F847871BD3D73F0AA43B6B0BB9BF',
 '08D4A53414F1B6629BCFB60498D1982DC027CE5267B47411B01DD7ADC77ECA17',
 '9819BF1E715263A59D781C1BCB5FB6036917955394017E3C97BE19336B8F753E',
 'E6D2C569927E0F5853FA138BF760B9F43A46B0F6ABB44B9684C40380DF67ADFD',
 'DF14E2B855218E0181AAB1E949F8C913

## 2.2 How many times account make transactions

We want to know it senders appears several times in transactions

In [1103]:
all_senders = df.sender.tolist() #all_senders is a list (with duplicates)

In [1104]:
for txn in tqdm(data):
    all_senders.append(txn["sender"])

In [1105]:
nb_txn_by_sender = Counter(all_senders).most_common() #count nb of occurences of each sender and sort them reverse

In [1107]:
set_senders = set(all_senders) #senders with no diplicates

In [1110]:
senders_with_at_least_2_txn = [ sender for sender, occurence in tqdm(list(nb_txn_by_sender)) if y > 1 ]

In [1111]:
print("There are", len(senders_with_at_least_2_txn)/len(set_senders)*100, "% of senders than appears at least two times")

There are 59.685003878942325 % of senders than appears at least two times


# 3. Trust lines

## 3.0 Getting trust lines (TODO parallelize)

In [200]:
all_senders = set(df.sender.tolist())

In [201]:
for txn in tqdm(data):
    all_senders.add(txn["sender"])

Next cell is very long

In [ ]:
lines = []
for address in tqdm(all_senders):
    answer = rpc.account_lines(address)
    if(answer["status"] == 'success'):
        lines.append(rpc.account_lines(address)) #account trust lines using Ripple's rpc method
    else:
        print("There was an error")

## 3.1 Trust lines with gateways

First we get the informations about the gateways with the ripple data API:

In [1070]:
gateways = api.get_all_gateways()
#online_gateways = ['SnapSwap','Gatehub Fifth','Bitstamp','Payroutes','WisePass','The Rock Trading','RippleChina','DotPayco','Coinex','Ripple Fox','Lake BTC','Bitso','GBI','Gatehub','EXRP','BPG']

In [1071]:
gateways_address = set() #all gateway addresses
#online_gateways_address = set()
for k,vs in tqdm(gateways.items()):
    for v in vs:
        gateways_address.add(v["account"])
        #if v['name'] in online_gateways:
            #online_gateways_address.add(v["account"])

We want :
* know what are the most popular wallet in trust lines
* Map wallet to countries given their trust lines currency 

In [1074]:
ctr_lines_with_gateways = 0 #counter for the number of trust lines with gateways
ctr_lines = 0 #counter for the number of trust lines
account_to_country = collections.defaultdict(list) #dict from account to countries (based on currency of its trust lines)
wallet_count_in_trust_lines = {} #count in how many trust lines a wallet appear
for info in tqdm(lines): 
    for line in info["lines"]:
        ctr_lines += 1
        wallet_count_in_trust_lines[line["account"]] = wallet_count_in_trust_lines.get(line["account"], 0) + 1
        if line["account"] in gateways_address:
            ctr_lines_with_gateways += 1
            
            #try to convert trust line currency into country, could fail if currency is not fiat (ex : BTC)
            try: 
                country = ccy.country(ccy.currency(line["currency"]).default_country)
                if(country not in account_to_country[info["account"]]):
                    account_to_country[info["account"]].append(country)
            except:
                continue

Now that we have collected the data we wanted, let's analyse it :
 * 3.1.1 What percentage of trust lines are with gateways
 * 3.1.2 Most Common wallets in trust lines 
 * 3.1.3 Over the accounts with trust lines, how many have one with a gateway
 * 3.1.4 Over all accounts, how many have a trust line with a gateway
 * 3.1.5 Currency of trust lines with gateways

### 3.1.1 What percentage of trust lines are with gateways

In [1093]:
print("There are", ctr_lines_with_gateways/ctr_lines*100,"% of thurst lines that are with gateways")

There are 64.61305374660225 % of thurst lines that are with gateways


### 3.1.2 Most Common wallets in trust lines 

In [1085]:
wallet_occurences = [(k,v) for k,v in list(wallet_count_in_trust_lines.items())] # convert dict to list of pairs

In [1086]:
wallet_occurences.sort(reverse = True,key=lambda wallet_occurences: wallet_occurences[1]) # sort by decressing occurence

In [1087]:
gateway_occurences = [(k,v) for k,v in x if k in gateways_address]  # get occurences in trust lines for gateways

In [1091]:
gateway_top = [i for i,j in zip(wallet_occurences,gateway_occurences) if i == j] # get first matching elements in occurence of wallets and gateways

In [1092]:
print("The",len(gateway_top),"most common wallet in trust lines are gateways")

The 6 most common wallet in trust lines are gateways


### 3.1.3 Over the accounts with trust lines, how many have one with a gateway

In [1095]:
nb_account_with_turst_lines = len([x for x in lines if len(x["lines"]) >= 1]) #nb of accounts with at least one trust line

In [1096]:
nb_account_with_trust_lines_with_gateways =len(account_to_country.keys()) #nb of accounts with trust line with gateways

In [1097]:
print("Over the accounts that have a trust line",nb_account_with_trust_lines_with_gateways/nb_account_with_turst_lines* 100,"% of them have one with a gateway.")

Over the accounts that have a trust line 70.74827222211158 % of them have one with a gateway.


### 3.1.4 Over all accounts, how many have a trust line with a gateway

In [1057]:
print("Over all accounts",nb_account_with_trust_lines_with_gateways/len(lines)*100,"% of them have a trust line with a gateway.")

Over all accounts 29.969542804593047 % of them have a trust line with a gateway.


### 3.1.5 Currency of trust lines with gateways

In [1100]:
count_len = {}
for country_list in tqdm(account_to_country.values()):
    # how many wallets are associated with "n" countries
    count_len[len(country_list)] = count_len.get(len(country_list),0) + 1 
for k,v in count_len.items():
    #percentage of total number of wallets that has trust lines with gateways
    count_len[k] = v/nb_account_with_trust_lines_with_gateways*100 

In [1101]:
print("Over the wallets that have trust lines with gateways",count_len[1],"% of them have trust lines with gateways in just 1 currency")
print("And",count_len[2],"% of them have trust lines with gateways in 2 currencies")

Over the wallets that have trust lines with gateways 75.11401385057148 % of them have trust lines with gateways in just 1 currency
And 19.373909126738358 % of them have trust lines with gateways in 2 currencies


## 3.2 How Wallet use their trust lines

We want to see if transactions goes mainly through trust lines with gateways

In [1162]:
wallet_to_gateway_trust_lines = collections.defaultdict(list)
for account_line in tqdm(lines):
    for line in account_line['lines']:
        if line['account'] in gateways_address:
            wallet = account_line['account']
            wallet_to_gateway_trust_lines[wallet].append((line['account'],line['currency']))

In [ ]:
usage_of_trust_lines = collections.defaultdict(dict)

# For a given currency transactions checks 
# if the senders uses one of its trust lines with gateways for this currency in the transaction
#     increment the counter for the this line
# else
#     if the sender uses a trust line that is not with a gateways OR makes a direct transaction in this currency
#         increment the counter for 'others'

def get_result(txn):
    if(txn['success'] == 'tesSUCCESS'):
        for account,trust_lines in wallet_to_gateway_trust_lines.items():
            if account == txn['sender']:
                
                #go first through the gateways 
                for trust_line in trust_lines:  
                    trust_line_currency = trust_line[1]
                    if 'links' in txn.keys():
                        for link in txn['links']:
                            if ((account == link['node1'] and trust_line[0] == link['node2'] and trust_line_currency == link['currency']) or (account == link['node2'] and trust_line[0] == link['node1'] and trust_line_currency == link['currency'])):
                                usage_of_trust_lines[account] = usage_of_trust_lines.get(account,{})
                                usage_of_trust_lines[account][trust_line_currency] = usage_of_trust_lines[account].get(trust_line_currency,{})
                                usage_of_trust_lines[account][trust_line_currency][trust_line[0]] = usage_of_trust_lines[account][trust_line_currency].get(trust_line[0],0) + 1
                                return
                            
                #if transaction does not go through a gateway look if it uses a other path             
                for trust_line in trust_lines:  
                    if 'links' in txn.keys():
                        for link in txn['links']:
                            if ((account == link['node1'] and trust_line_currency == link['currency']) or (account == link['node2'] and trust_line_currency == link['currency'])):
                                usage_of_trust_lines[account] = usage_of_trust_lines.get(account,{})
                                usage_of_trust_lines[account][trust_line_currency] = usage_of_trust_lines[account].get(trust_line_currency,{})
                                usage_of_trust_lines[account][trust_line_currency]['other'] =  usage_of_trust_lines[account][trust_line_currency].get('other',0) + 1
                                return
                            
                    #or if it's direct                  
                    elif 'senderCurrency' in txn.keys():
                        if txn['senderCurrency'] == trust_line_currency:
                            usage_of_trust_lines[account] = usage_of_trust_lines.get(account,{})
                            usage_of_trust_lines[account][trust_line_currency] = usage_of_trust_lines[account].get(trust_line_currency,{})
                            usage_of_trust_lines[account][trust_line_currency]['other'] = usage_of_trust_lines[account][trust_line_currency].get('other',0) + 1
                            return
                        
#go through all transactions
for txn in tqdm(data):
    get_result(txn)           

Now count how many txn goes through gateways and how many does not.

In [1024]:
count_through_gateways = 0
count_others = 0
for account, currencies in usage_of_trust_lines.items():
    for currency,values in currencies.items():
        for key,value in values.items(): #key will be a gateway address or 'other'
            if key == 'other':
                count_others += value
            else:
                count_through_gateways += value

In [1025]:
print("For wallets that have trust line with gateways in a given currency, there are in average", count_through_gateways/(count_through_gateways+count_others)*100,'% of payments in that currency made by this wallet via these trust lines with gateways') 

For wallets that have trust line with gateways in a given currency, there are in average 98.91175878891515 % of payments in that currency made by this wallet via these trust lines with gateways


# 4. Gateways

## 4.1 Gateway per currency zone

Our goal here is to find what currencies are covered by gateways and how many gateways can we find by country.

In [1129]:
dict_gateways = api.get_all_gateways()

In [1130]:
print("There are",len(list(dict_gateways.values())),"known gateways.")

There are 34 known gateways.


List of online gateways according to https://wipple.devnull.network/research/gateways.html

In [1131]:
online_gateways = ['SnapSwap','Gatehub Fifth','Bitstamp','Payroutes','WisePass','The Rock Trading','RippleChina','DotPayco','Coinex','Ripple Fox','Lake BTC','Bitso','GBI','Gatehub','EXRP','BPG']

In [1132]:
currency_to_nb_gateways = {}
for currency, gateways in tqdm(dict_gateways.items()):
    for gateway in gateways:
        if gateway['name'] in online_gateways:
            currency_to_nb_gateways[currency] = currency_to_nb_gateways.get(currency,0) + 1

In [1133]:
country_to_nb_gateways= {}
for currency, nb_gateways in tqdm(currency_to_nb_gateways.items()):
    #Try to convert currency to country (only works if currency is real, ex: BTC will fail)
    try:
        currency = ccy.country(ccy.currency(currency).default_country)
        country_to_nb_gateways[currency] = nb_gateways
    except :
        continue

In [1134]:
country_to_nb_gateways

{'Australia': 2,
 'Switzerland': 1,
 'Eurozone': 4,
 'Britain (UK)': 3,
 'Japan': 2,
 'United States': 8,
 'China': 3,
 'Israel': 1,
 'Norway': 1,
 'Sweden': 1,
 'Mexico': 1,
 'New Zealand': 1,
 'Korea (South)': 1}